In [112]:
import pandas as pd
import json


In [113]:
estimate_df = pd.read_csv("/content/estimates-clean - dataset.csv")
orderitem_df = pd.read_csv("/content/order-item-clean.csv")

In [114]:
# prompt: get count of rows of orderitemdf

orderitemdf_rows = estimate_df.shape[0]
print(orderitemdf_rows)

estimateddf_rows = orderitem_df.shape[0]
print(estimateddf_rows)

87670
48246


In [115]:
# prompt: show me data in estimatedf where orderId = 12488 and orderItemId = 51200

estimate_df.loc[(estimate_df['orderId'] == 12488) & (estimate_df['orderItemId'] == 51200)]


,created_at,orderId,orderItemId,brandId,productName,estimatedDeliveryStartDate,estimatedDeliveryEndDate,variant,quantity,msrp,discount,msrpTotal,totalCost,deliveryFees,salesTaxAmount,totalSavedAmount,companyId
11,2023-10-24T16:55:25.988Z,12488,51200,165,Jute Pouf,2023-12-19,2024-01-09,Neutral // One Size,1.0,199.95,0.0,1744.0,1663.62,249.0,97.86,427.24,6060
14,2023-10-19T16:21:24.792Z,12488,51200,165,Jute Pouf,2023-11-02,2023-11-23,Neutral // One Size,1.0,199.95,0.0,2542.0,2341.92,249.0,137.76,586.84,6060


In [76]:
estimate_df['companyId'] = df['companyId'].astype(str)


In [116]:
# prompt: in estimate_df give me count of deliveredOnDate where its not blank

delivered_on_date_count = orderitem_df['deliveredOnDate'].notna().sum()
print(delivered_on_date_count)
#11243

11243


In [119]:
# prompt: for estimatedf, keep all columns of the df and for each unique order_item_id take the max(created_at value)
# then as a check show me the row where orderId = '12488' and orderitemId = 51200

# Group by orderItemId and select the row with the maximum createdAt value
max_created_at_df = estimate_df.groupby('orderItemId')['created_at'].max().reset_index()

# Keep all columns of the original DataFrame
max_created_at_df = max_created_at_df.merge(estimate_df, on=['orderItemId', 'created_at'])



In [120]:
len(max_created_at_df)

53157

In [121]:
# prompt: show columns of estimateddf

print(max_created_at_df.columns.tolist())


['orderItemId', 'created_at', 'orderId', 'brandId', 'productName', 'estimatedDeliveryStartDate', 'estimatedDeliveryEndDate', 'variant', 'quantity', 'msrp', 'discount', 'msrpTotal', 'totalCost', 'deliveryFees', 'salesTaxAmount', 'totalSavedAmount', 'companyId']


In [122]:
# prompt: AS A SPOT CHECK SHOW ME ROW WHERE ORDERITEMID = 51200 in max_created_at_df

max_created_at_df.loc[max_created_at_df['orderItemId'] == 51200]


,orderItemId,created_at,orderId,brandId,productName,estimatedDeliveryStartDate,estimatedDeliveryEndDate,variant,quantity,msrp,discount,msrpTotal,totalCost,deliveryFees,salesTaxAmount,totalSavedAmount,companyId
15,51200,2023-10-24T16:55:25.988Z,12488,165,Jute Pouf,2023-12-19,2024-01-09,Neutral // One Size,1.0,199.95,0.0,1744.0,1663.62,249.0,97.86,427.24,6060


In [123]:
estimate_df = max_created_at_df

In [132]:
# prompt: count number of order_item_ids in both dfs

orderitem_ids_in_estimate_df = estimate_df['orderItemId'].nunique()
orderitem_ids_in_orderitem_df = orderitem_df['orderItemId'].nunique()

print(f"Number of order_item_ids in estimate_df: {orderitem_ids_in_estimate_df}")
print(f"Number of order_item_ids in orderitem_df: {orderitem_ids_in_orderitem_df}")


Number of order_item_ids in estimate_df: 53157
Number of order_item_ids in orderitem_df: 48246


In [134]:
# prompt: create a new df make it a left join of the two dfs with overitem_df on the left. join on orderId and orderItemId. show me row count of new df. show me count of deliveredOnDate

left_joined_df = pd.merge(orderitem_df, estimate_df, how='left', on=['orderId', 'orderItemId'])

print(f"Number of rows in left_joined_df: {len(left_joined_df)}")

delivered_on_date_count = left_joined_df['deliveredOnDate'].notna().sum()
print(f"Number of rows with deliveredOnDate not null: {delivered_on_date_count}")


Number of rows in left_joined_df: 48246
Number of rows with deliveredOnDate not null: 11243


In [136]:
# prompt: show me count of rows in new df where productName_x is like test

test_product_count = left_joined_df['productName_x'].str.contains('test', case=False, na=False).sum()
print(f"Number of rows with 'test' in productName_x: {test_product_count}")


Number of rows with 'test' in productName_x: 254


In [137]:
df = left_joined_df


In [57]:
# Remove rows where productName_x is like 'test product'
df = df[df['productName_x'].str.contains('test product') == False]
# Remove rows where productName_x is like 'test product'
df = df[df['productName_x'].str.contains('test') == False]
df = df[df['productName_x'] != 'way']

# If you want to reset the index after dropping rows
df.reset_index(drop=True, inplace=True)


In [188]:
df['actualDeliveryStartDate'] = pd.to_datetime(df['actualDeliveryStartDate'])
df['actualDeliveryEndDate'] = pd.to_datetime(df['actualDeliveryEndDate'])
df['estimatedDeliveryStartDate'] = pd.to_datetime(df['estimatedDeliveryStartDate'])
df['estimatedDeliveryEndDate'] = pd.to_datetime(df['estimatedDeliveryEndDate'])
df['deliveredOnDate'] = pd.to_datetime(df['deliveredOnDate'])

In [141]:
df


,orderItemId,orderId,brandId_x,actualDeliveryStartDate,actualDeliveryEndDate,deliveredOnDate,msrp_x,productName_x,quantity_x,variant_x,...,msrp_y,discount,msrpTotal,totalCost,deliveryFees,salesTaxAmount,totalSavedAmount,companyId,deliveryFeesPerItem,AccurateDelivery
0,226921,37396,91,NaT,NaT,NaT,179.99,Cojocar Geometric Rug,1,Rectangle 6' x 9',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,123209,29388,208,2023-09-13,2023-09-15,NaT,148.00,Entryway Rack,1,Navy,...,148.00,0.0,929.50,720.78,0.0,62.53,271.25,12272.0,0.000000,False
2,122772,29311,91,2023-09-15,2023-09-15,NaT,139.67,Coffee Table,1,Teak,...,139.67,0.0,2736.52,1981.91,0.0,176.07,930.68,12785.0,0.000000,True
3,120737,28739,91,2023-09-11,2023-09-11,NaT,81.99,Kaniel Metal USB Table Lamp,2,Brass/Gold,...,81.99,0.0,2700.94,1413.95,0.0,118.90,1405.89,6202.0,0.000000,True
4,123070,29358,91,2023-09-18,2023-09-18,NaT,32.99,Alicia Scalloped 52'' W Window Valance,2,Rouge,...,32.99,0.0,4802.56,2892.89,33.4,168.21,2111.28,1708.0,2.226667,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48241,138460,33324,310,NaT,NaT,NaT,76.30,Pfister Tenet Wall Mounted Pivoting Toilet Pap...,1,Polished Nickel,...,74.01,0.0,1664.25,1470.24,0.0,92.97,286.98,2112.0,0.000000,False
48242,138458,33324,310,NaT,NaT,NaT,111.36,Pfister 1/2 Inch Mixing Rough In Valve with Se...,1,Ceramic Disc Valve,...,103.05,0.0,1664.25,1470.24,0.0,92.97,286.98,2112.0,0.000000,False
48243,138463,33324,310,NaT,NaT,NaT,317.90,Kohler Cimarron 1.28 GPF Two Piece Elongated C...,1,White // Elongated // Left,...,234.15,0.0,1664.25,1470.24,0.0,92.97,286.98,2112.0,0.000000,False
48244,138457,33324,310,NaT,NaT,NaT,646.80,Pfister Tenet Pressure Balanced Shower System ...,1,Brushed Nickel,...,627.40,0.0,1664.25,1470.24,0.0,92.97,286.98,2112.0,0.000000,False


In [145]:
df['brandId_x'] = df['brandId_x'].astype(str)

In [173]:
import pandas as pd

# Calculate if actual delivery falls within the EDD range
df['AccurateDelivery'] = (
    (df['actualDeliveryStartDate'] >= df['estimatedDeliveryStartDate']) &
    (df['actualDeliveryEndDate'] <= df['estimatedDeliveryEndDate'])
)

# Group by BrandID and Category, and calculate accuracy metrics
accuracy_metrics = df.groupby(['brandId_x']).agg(
    AccurateDeliveries=('AccurateDelivery', 'sum'),
    TotalOrders=('brandId_x', 'count')
)

# Calculate accuracy percentage
accuracy_metrics['AccuracyPercentage'] = (accuracy_metrics['AccurateDeliveries'] / accuracy_metrics['TotalOrders']) * 100
accuracy_metrics


,AccurateDeliveries,TotalOrders,AccuracyPercentage
brandId_x,,,
0,0,1,0.000000
10,98,225,43.555556
109,10,27,37.037037
11,24,113,21.238938
111,20,60,33.333333
...,...,...,...
86,9,54,16.666667
9,41,277,14.801444
90,17,179,9.497207


In [178]:
import pandas as pd

# Calculate if actual delivery falls within the EDD range
df['AccurateDelivery'] = (
    (df['actualDeliveryStartDate'] >= df['estimatedDeliveryStartDate']) &
    (df['actualDeliveryEndDate'] <= df['estimatedDeliveryEndDate'])
)

# Group by BrandID and Category, and calculate accuracy metrics
accuracy_metrics = df.groupby(['brandId_x']).agg(
    AccurateDeliveries=('AccurateDelivery', 'sum'),
    TotalOrders=('brandId_x', 'count')
)

# Calculate accuracy percentage
accuracy_metrics['AccuracyPercentage'] = (accuracy_metrics['AccurateDeliveries'] / accuracy_metrics['TotalOrders']) * 100

accuracy_metrics_sorted = accuracy_metrics.sort_values(by='TotalOrders', ascending=False)

print(accuracy_metrics_sorted)



           AccurateDeliveries  TotalOrders  AccuracyPercentage
brandId_x                                                     
91                       7445        18030           41.292291
32                        953         3990           23.884712
37                        781         3313           23.573800
82                        891         2781           32.038835
310                       460         1720           26.744186
...                       ...          ...                 ...
383                         0            1            0.000000
523                         1            1          100.000000
498                         0            1            0.000000
435                         0            1            0.000000
0                           0            1            0.000000

[159 rows x 3 columns]


In [183]:
import pandas as pd

# Assuming accuracy_metrics_sorted is your DataFrame containing accuracy metrics sorted by TotalOrders
# and you have a datetime column named 'deliveredOnDate' representing the delivery date

# Filter data for all months in 2023
accuracy_metrics_2023 = accuracy_metrics_sorted.loc[accuracy_metrics_sorted['deliveredOnDate'].dt.year == 2023]

# Select the top 10 brands
top_10_brands_2023 = accuracy_metrics_2023.head(10)

print(top_10_brands_2023)



KeyError: 'deliveredOnDate'

In [161]:
# prompt: create a new column called Order where if deliveredOndate is blank the value is 0

df['Order'] = df['deliveredOnDate'].apply(lambda x: 0 if pd.isna(x) else 1)


In [164]:
# prompt: create a new df deliveredOrders which is filtered by Order = 1 and write to csv

deliveredOrders = df[df['Order'] == 1]
deliveredOrders.to_csv('deliveredOrders.csv', index=False)
deliveredOrders.to_csv("/content/drive/MyDrive/deliveredOrders.csv")


In [165]:
# Create fields
df['deltaStartDate'] = (df['actualDeliveryStartDate'] - df['estimatedDeliveryStartDate']).dt.days
df['deltaEndDate'] = (df['actualDeliveryEndDate'] - df['estimatedDeliveryEndDate']).dt.days
df['deliveredOnDate-actualDeliveryEndDate'] = (df['deliveredOnDate'] - df['actualDeliveryEndDate']).dt.days


In [171]:
# prompt: count deliveredOnDate is not blank in df

delivered_on_date_count = df['deliveredOnDate'].notna().sum()
print(f"Number of rows with deliveredOnDate not null: {delivered_on_date_count}")


Number of rows with deliveredOnDate not null: 11243


In [186]:
df

,Unnamed: 0,orderItemId,orderId,brandId_x,actualDeliveryStartDate,actualDeliveryEndDate,deliveredOnDate,msrp_x,productName_x,quantity_x,...,deliveryFees,salesTaxAmount,totalSavedAmount,companyId,deliveryFeesPerItem,AccurateDelivery,Order,deltaStartDate,deltaEndDate,deliveredOnDate-actualDeliveryEndDate
0,0,226921,37396,91,NaN,NaN,NaN,179.99,Cojocar Geometric Rug,1,...,NaN,NaN,NaN,NaN,NaN,False,0,NaN,NaN,NaN
1,1,123209,29388,208,2023-09-13,2023-09-15,NaN,148.00,Entryway Rack,1,...,0.0,62.53,271.25,12272.0,0.000000,False,0,-5.0,-17.0,NaN
2,2,122772,29311,91,2023-09-15,2023-09-15,NaN,139.67,Coffee Table,1,...,0.0,176.07,930.68,12785.0,0.000000,True,0,1.0,-6.0,NaN
3,3,120737,28739,91,2023-09-11,2023-09-11,NaN,81.99,Kaniel Metal USB Table Lamp,2,...,0.0,118.90,1405.89,6202.0,0.000000,True,0,2.0,-5.0,NaN
4,4,123070,29358,91,2023-09-18,2023-09-18,NaN,32.99,Alicia Scalloped 52'' W Window Valance,2,...,33.4,168.21,2111.28,1708.0,2.226667,True,0,3.0,-4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48241,48241,138460,33324,310,NaN,NaN,NaN,76.30,Pfister Tenet Wall Mounted Pivoting Toilet Pap...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48242,48242,138458,33324,310,NaN,NaN,NaN,111.36,Pfister 1/2 Inch Mixing Rough In Valve with Se...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48243,48243,138463,33324,310,NaN,NaN,NaN,317.90,Kohler Cimarron 1.28 GPF Two Piece Elongated C...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48244,48244,138457,33324,310,NaN,NaN,NaN,646.80,Pfister Tenet Pressure Balanced Shower System ...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN


In [166]:
df.to_csv("/content/drive/MyDrive/minoan_final_analysis2.csv")

In [191]:
df

,Unnamed: 0,orderItemId,orderId,brandId_x,actualDeliveryStartDate,actualDeliveryEndDate,deliveredOnDate,msrp_x,productName_x,quantity_x,...,deliveryFees,salesTaxAmount,totalSavedAmount,companyId,deliveryFeesPerItem,AccurateDelivery,Order,deltaStartDate,deltaEndDate,deliveredOnDate-actualDeliveryEndDate
0,0,226921,37396,91,NaN,NaN,NaN,179.99,Cojocar Geometric Rug,1,...,NaN,NaN,NaN,NaN,NaN,False,0,NaN,NaN,NaN
1,1,123209,29388,208,2023-09-13,2023-09-15,NaN,148.00,Entryway Rack,1,...,0.0,62.53,271.25,12272.0,0.000000,False,0,-5.0,-17.0,NaN
2,2,122772,29311,91,2023-09-15,2023-09-15,NaN,139.67,Coffee Table,1,...,0.0,176.07,930.68,12785.0,0.000000,True,0,1.0,-6.0,NaN
3,3,120737,28739,91,2023-09-11,2023-09-11,NaN,81.99,Kaniel Metal USB Table Lamp,2,...,0.0,118.90,1405.89,6202.0,0.000000,True,0,2.0,-5.0,NaN
4,4,123070,29358,91,2023-09-18,2023-09-18,NaN,32.99,Alicia Scalloped 52'' W Window Valance,2,...,33.4,168.21,2111.28,1708.0,2.226667,True,0,3.0,-4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48241,48241,138460,33324,310,NaN,NaN,NaN,76.30,Pfister Tenet Wall Mounted Pivoting Toilet Pap...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48242,48242,138458,33324,310,NaN,NaN,NaN,111.36,Pfister 1/2 Inch Mixing Rough In Valve with Se...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48243,48243,138463,33324,310,NaN,NaN,NaN,317.90,Kohler Cimarron 1.28 GPF Two Piece Elongated C...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN
48244,48244,138457,33324,310,NaN,NaN,NaN,646.80,Pfister Tenet Pressure Balanced Shower System ...,1,...,0.0,92.97,286.98,2112.0,0.000000,False,0,NaN,NaN,NaN


In [204]:
# prompt: convert deliveryOndate to date

df['deliveredOnDate'] = pd.to_datetime(df['deliveredOnDate'])

df['actualDeliveryEndDate'] = pd.to_datetime(df['actualDeliveryEndDate'])

df['actualDeliveryStartDate'] = pd.to_datetime(df['actualDeliveryStartDate'])

df['estimatedDeliveryStartDate'] = pd.to_datetime(df['estimatedDeliveryStartDate'])
df['estimatedDeliveryEndDate'] = pd.to_datetime(df['estimatedDeliveryEndDate'])

In [208]:
# prompt: remove NaNs from date columns and predict Accurate Delivery using a log reg model to forecast delivery accuracy. Use other features too like totalCost, Shipping cost

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data
df

# Drop rows with NaN values in date columns
df.dropna(subset=['deliveredOnDate', 'actualDeliveryEndDate', 'actualDeliveryStartDate', 'estimatedDeliveryStartDate', 'estimatedDeliveryEndDate'], inplace=True)

# Select features for prediction
features = ['totalCost', 'deliveryFees', 'deltaStartDate', 'deltaEndDate', 'deliveredOnDate-actualDeliveryEndDate']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df['AccurateDelivery'], test_size=0.25, random_state=42)

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7524659464537341


In [213]:

import pandas as pd
import statsmodels.api as sm


# Drop rows with NaN values in date columns
df.dropna(subset=['deliveredOnDate', 'actualDeliveryEndDate', 'actualDeliveryStartDate', 'estimatedDeliveryStartDate', 'estimatedDeliveryEndDate'], inplace=True)

# Select features for prediction
features = ['totalCost', 'deliveryFees', 'deltaStartDate', 'deltaEndDate', 'deliveredOnDate-actualDeliveryEndDate']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[features], df['AccurateDelivery'], test_size=0.25, random_state=42)

# Train the Logistic Regression model
model = sm.Logit(y_train, X_train)
result = model.fit()

# Get the p-values of the model coefficients
p_values = result.pvalues

# Select features with p-value less than 0.05
significant_features = p_values[p_values < 0.05].index.tolist()

# Print the significant features
print("Significant features:", significant_features)

# Print the p-value of the model
print("Model p-value:", result.pvalues[0])

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Optimization terminated successfully.
         Current function value: 0.619807
         Iterations 6
Significant features: ['totalCost', 'deliveryFees', 'deltaStartDate', 'deltaEndDate', 'deliveredOnDate-actualDeliveryEndDate']
Model p-value: 3.7647047523604935e-08
Accuracy: 0.7524659464537341


TypeError: could not convert string to float: '2023-09-14'